In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

%matplotlib inline

In [0]:
import os

path = 'drive/My Drive/Projects/Uma.Tech_task'
os.chdir(path)

### Загружаем данные

In [0]:
train_raw_data = pd.read_csv('data/TRAIN_RAW_DATA.csv', parse_dates=['calday', 'dt1'])
train_prepared = pd.read_csv('data/TRAIN_PREPARED.csv', parse_dates=['cut_date', 'first_date', 'last_date'])
train_additional_data = pd.read_csv('data/TRAIN_ADDITIONAL_DATA.csv')

test_raw_data = pd.read_csv('data/TEST_RAW_DATA.csv', parse_dates=['calday', 'dt1'])
test_prepared = pd.read_csv('data/TEST_PREPARED.csv', parse_dates=['cut_date', 'first_date', 'last_date'])
test_additional_data = pd.read_csv('data/TEST_ADDITIONAL_DATA.csv')

In [4]:
train_raw_data.head()

,email,title_ep,title_ru,calday,dt1,timelength,device,country_iso_code,city_iso_code,utc_offset
0,513ed17481,1821f1a1ce58,f1bcc7a5177e,2019-03-04,2019-03-04 17:14:48,45,PC,RU,MOW,3.0
1,0160e02b90,c724f041dc39,a714d9f17666,2019-03-04,2019-03-04 23:23:19,60,AndroidPhone_PlayReady_HW,RU,MOW,3.0
2,0160e02b90,50409694c8a8,5e8ec479ee7e,2019-03-04,2019-03-04 21:06:59,1273,AndroidPhone_PlayReady_HW,RU,MOW,3.0
3,8a92686302,c724f041dc39,a714d9f17666,2019-03-04,2019-03-04 23:46:01,60,PC,RU,MOW,3.0
4,8a92686302,c724f041dc39,a714d9f17666,2019-03-04,2019-03-04 23:46:05,60,PC,RU,MOW,3.0


In [0]:
some_email = train_prepared.email.unique()[1]

train_prepared[train_prepared['email'] == some_email].head()

,cut_date,days_to_end,email,first_date,last_date,num_country_max_1days,num_city_max_1days,android_max_1days,smarttv_max_1days,iphone_max_1days,ipad_max_1days,apple_max_1days,pc_max_1days,time_spent_sum_3days,time_spent_max_3days,num_title_ep_sum_3days,num_title_ep_max_3days,num_title_ru_sum_3days,num_title_ru_max_3days,num_country_max_3days,num_city_max_3days,other_content_sum_3days,top_1_sum_3days,top_2_sum_3days,top_3_sum_3days,top_4_sum_3days,top_5_sum_3days,top_6_sum_3days,top_7_sum_3days,top_8_sum_3days,top_9_sum_3days,top_10_sum_3days,android_max_3days,smarttv_max_3days,iphone_max_3days,ipad_max_3days,apple_max_3days,pc_max_3days,time_spent_sum_7days,time_spent_max_7days,...,top_10_sum_21days,android_max_21days,smarttv_max_21days,iphone_max_21days,ipad_max_21days,apple_max_21days,pc_max_21days,time_spent_sum_30days,time_spent_max_30days,num_title_ep_sum_30days,num_title_ep_max_30days,num_title_ru_sum_30days,num_title_ru_max_30days,num_country_max_30days,num_city_max_30days,other_content_sum_30days,top_1_sum_30days,top_2_sum_30days,top_3_sum_30days,top_4_sum_30days,top_5_sum_30days,top_6_sum_30days,top_7_sum_30days,top_8_sum_30days,top_9_sum_30days,top_10_sum_30days,android_max_30days,smarttv_max_30days,iphone_max_30days,ipad_max_30days,apple_max_30days,pc_max_30days,activity_1to3,activity_1to7,activity_1to14,activity_7to14,activity_7to21,activity_7to30,activity_14to30,label
1,2019-04-02,10,752a6d96f7,2018-10-28,2019-04-01 20:51:29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,60.0,60.0,...,0.0,0.0,0.0,11.0,0.0,0.0,1.0,19801.0,10152.0,22.0,9.0,10.0,3.0,1.0,11.0,1.0,2.0,10.0,9.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0,0.005807,0.005774,0.003030,0.521792,1.0
10441,2019-03-30,13,752a6d96f7,2018-10-28,2019-03-26 09:35:23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,120.0,...,0.0,0.0,0.0,11.0,0.0,0.0,0.0,28713.0,10152.0,24.0,9.0,11.0,3.0,1.0,11.0,1.0,4.0,10.0,9.0,1.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0,0.011682,0.006149,0.004179,0.357747,1.0
11055,2019-04-07,5,752a6d96f7,2018-10-28,2019-04-01 20:51:29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,...,0.0,0.0,0.0,11.0,0.0,0.0,1.0,19576.0,10152.0,19.0,9.0,9.0,3.0,1.0,11.0,1.0,2.0,6.0,9.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.333315,0.005807,0.003065,0.009195,1.0
26853,2019-04-04,8,752a6d96f7,2018-10-28,2019-04-01 20:51:29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,...,0.0,0.0,0.0,11.0,0.0,0.0,1.0,19576.0,10152.0,19.0,9.0,9.0,3.0,1.0,11.0,1.0,2.0,6.0,9.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.005807,0.005807,0.003065,0.527789,1.0
34228,2019-03-29,14,752a6d96f7,2018-10-28,2019-03-26 09:35:23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10272.0,10152.0,...,0.0,0.0,0.0,11.0,0.0,0.0,0.0,28713.0,10152.0,24.0,9.0,11.0,3.0,1.0,11.0,1.0,4.0,10.0,9.0,1.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0,0.999999,0.526337,0.357747,0.357747,1.0


In [0]:
# Доля объектов из класса 1 -> классы примерно сбалансированы

train_prepared.groupby(['email'])['label'].mean().mean()

0.27686333333333335

### First baseline

In [0]:
# Process time features

def get_time_features(df, column):
    df[column + '_month'] = df[column].dt.month
    df[column + '_week'] = df[column].dt.week
    df[column + '_day'] = df[column].dt.day
    df[column + '_dow'] = df[column].dt.dayofweek
    df[column + '_hour'] = df[column].dt.hour
    df[column + '_minute'] = df[column].dt.minute

    df.drop([column], axis=1, inplace=True)

get_time_features(train_prepared, 'cut_date')
get_time_features(train_prepared, 'first_date')
get_time_features(train_prepared, 'last_date')

In [0]:
# Process categorial feature

le = preprocessing.LabelEncoder()

le.fit(train_prepared['email'])
train_prepared['email'] = le.transform(train_prepared['email'])

In [0]:
all_emails = train_prepared['email'].unique()

emails_train = all_emails[:14000]
emails_test = all_emails[14000:]

X_train = train_prepared[train_prepared['email'].isin(emails_train)].drop(['label'], axis=1)
y_train = train_prepared[train_prepared['email'].isin(emails_train)]['label']

X_test = train_prepared[train_prepared['email'].isin(emails_test)].drop(['label'], axis=1)
y_test = train_prepared[train_prepared['email'].isin(emails_test)]['label']

In [0]:
X_train['email'].unique()

array([10848,  9121, 10137, ..., 18293,  6709, 12893])

In [0]:
clf = LGBMClassifier(n_estimators=300, learning_rate=0.1)
clf.fit(X_train, y_train)
    
predictions = clf.predict_proba(X_test)
X_test['predictions'] = predictions[:, 1]
X_test['y'] = y_test

In [0]:
y_pred = X_test.groupby(['email'])[['predictions', 'y']].mean()['predictions']
y_true = X_test.groupby(['email'])[['predictions', 'y']].mean()['y']

auc_score = roc_auc_score((y_true > 0).values, y_pred.values)
print(auc_score)

0.63064383495506


### Временные признаки

In [0]:
train_prepared = pd.read_csv('data/TRAIN_PREPARED.csv', parse_dates=['cut_date', 'first_date', 'last_date'])

In [6]:
some_email = train_prepared.email.unique()[2]

train_prepared[train_prepared['email'] == some_email].sort_values(by=['cut_date']).head()

,cut_date,days_to_end,email,first_date,last_date,num_country_max_1days,num_city_max_1days,android_max_1days,smarttv_max_1days,iphone_max_1days,ipad_max_1days,apple_max_1days,pc_max_1days,time_spent_sum_3days,time_spent_max_3days,num_title_ep_sum_3days,num_title_ep_max_3days,num_title_ru_sum_3days,num_title_ru_max_3days,num_country_max_3days,num_city_max_3days,other_content_sum_3days,top_1_sum_3days,top_2_sum_3days,top_3_sum_3days,top_4_sum_3days,top_5_sum_3days,top_6_sum_3days,top_7_sum_3days,top_8_sum_3days,top_9_sum_3days,top_10_sum_3days,android_max_3days,smarttv_max_3days,iphone_max_3days,ipad_max_3days,apple_max_3days,pc_max_3days,time_spent_sum_7days,time_spent_max_7days,...,top_10_sum_21days,android_max_21days,smarttv_max_21days,iphone_max_21days,ipad_max_21days,apple_max_21days,pc_max_21days,time_spent_sum_30days,time_spent_max_30days,num_title_ep_sum_30days,num_title_ep_max_30days,num_title_ru_sum_30days,num_title_ru_max_30days,num_country_max_30days,num_city_max_30days,other_content_sum_30days,top_1_sum_30days,top_2_sum_30days,top_3_sum_30days,top_4_sum_30days,top_5_sum_30days,top_6_sum_30days,top_7_sum_30days,top_8_sum_30days,top_9_sum_30days,top_10_sum_30days,android_max_30days,smarttv_max_30days,iphone_max_30days,ipad_max_30days,apple_max_30days,pc_max_30days,activity_1to3,activity_1to7,activity_1to14,activity_7to14,activity_7to21,activity_7to30,activity_14to30,label
149479,2019-03-29,8,827f6afef3,2018-12-14,2019-03-27 00:13:20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,833.0,833.0,2.0,2.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,893.0,833.0,...,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1493.0,833.0,12.0,5.0,11.0,5.0,1.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.000000,0.000000,0.000000,0.999989,0.832238,0.598121,0.598121,0.0
168396,2019-03-30,7,827f6afef3,2018-12-14,2019-03-27 00:13:20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,893.0,833.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1433.0,833.0,11.0,5.0,10.0,5.0,1.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.000000,0.000000,0.000000,0.999989,0.881531,0.623164,0.623164,0.0
124920,2019-03-31,6,827f6afef3,2018-12-14,2019-03-27 00:13:20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,893.0,833.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1433.0,833.0,11.0,5.0,10.0,5.0,1.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.000000,0.000000,0.000000,0.999989,0.881531,0.623164,0.623164,0.0
45052,2019-04-01,5,827f6afef3,2018-12-14,2019-04-01 18:00:49,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,953.0,833.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1493.0,833.0,12.0,5.0,11.0,5.0,1.0,6.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.999833,0.062958,0.062958,0.999990,0.888156,0.638308,0.638308,0.0
2,2019-04-02,4,827f6afef3,2018-12-14,2019-04-01 18:00:49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,893.0,833.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1493.0,833.0,12.0,5.0,11.0,5.0,1.0,6.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.000000,0.000000,0.000000,0.937031,0.832238,0.598121,0.638308,0.0


In [0]:
def get_time_features(df):

    df['days_use'] = (df['last_date'] - df['first_date']).dt.days
    df['days_no_use'] = (df['cut_date'] - df['last_date']).dt.days + 1

    df.drop(['last_date'], axis=1, inplace=True)
    df.drop(['first_date'], axis=1, inplace=True)
    # df.drop(['cut_date'], axis=1, inplace=True)

In [0]:
# Для тренировочного датасета
get_time_features(train_prepared)

# Для тестового датасета
get_time_features(test_prepared)

In [10]:
train_prepared[train_prepared['email'] == some_email].sort_values(by=['cut_date'])

,cut_date,days_to_end,email,num_country_max_1days,num_city_max_1days,android_max_1days,smarttv_max_1days,iphone_max_1days,ipad_max_1days,apple_max_1days,pc_max_1days,time_spent_sum_3days,time_spent_max_3days,num_title_ep_sum_3days,num_title_ep_max_3days,num_title_ru_sum_3days,num_title_ru_max_3days,num_country_max_3days,num_city_max_3days,other_content_sum_3days,top_1_sum_3days,top_2_sum_3days,top_3_sum_3days,top_4_sum_3days,top_5_sum_3days,top_6_sum_3days,top_7_sum_3days,top_8_sum_3days,top_9_sum_3days,top_10_sum_3days,android_max_3days,smarttv_max_3days,iphone_max_3days,ipad_max_3days,apple_max_3days,pc_max_3days,time_spent_sum_7days,time_spent_max_7days,num_title_ep_sum_7days,num_title_ep_max_7days,...,smarttv_max_21days,iphone_max_21days,ipad_max_21days,apple_max_21days,pc_max_21days,time_spent_sum_30days,time_spent_max_30days,num_title_ep_sum_30days,num_title_ep_max_30days,num_title_ru_sum_30days,num_title_ru_max_30days,num_country_max_30days,num_city_max_30days,other_content_sum_30days,top_1_sum_30days,top_2_sum_30days,top_3_sum_30days,top_4_sum_30days,top_5_sum_30days,top_6_sum_30days,top_7_sum_30days,top_8_sum_30days,top_9_sum_30days,top_10_sum_30days,android_max_30days,smarttv_max_30days,iphone_max_30days,ipad_max_30days,apple_max_30days,pc_max_30days,activity_1to3,activity_1to7,activity_1to14,activity_7to14,activity_7to21,activity_7to30,activity_14to30,label,days_use,days_no_use
149479,2019-03-29,8,827f6afef3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,833.0,833.0,2.0,2.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,893.0,833.0,3.0,2.0,...,0.0,0.0,0.0,0.0,2.0,1493.0,833.0,12.0,5.0,11.0,5.0,1.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.000000,0.000000,0.000000,0.999989,0.832238,0.598121,0.598121,0.0,103,2
168396,2019-03-30,7,827f6afef3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,893.0,833.0,3.0,2.0,...,0.0,0.0,0.0,0.0,2.0,1433.0,833.0,11.0,5.0,10.0,5.0,1.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.000000,0.000000,0.000000,0.999989,0.881531,0.623164,0.623164,0.0,103,3
124920,2019-03-31,6,827f6afef3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,893.0,833.0,3.0,2.0,...,0.0,0.0,0.0,0.0,2.0,1433.0,833.0,11.0,5.0,10.0,5.0,1.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.000000,0.000000,0.000000,0.999989,0.881531,0.623164,0.623164,0.0,103,4
45052,2019-04-01,5,827f6afef3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,953.0,833.0,4.0,2.0,...,0.0,0.0,0.0,0.0,2.0,1493.0,833.0,12.0,5.0,11.0,5.0,1.0,6.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.999833,0.062958,0.062958,0.999990,0.888156,0.638308,0.638308,0.0,108,0
2,2019-04-02,4,827f6afef3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,893.0,833.0,3.0,2.0,...,0.0,0.0,0.0,0.0,2.0,1493.0,833.0,12.0,5.0,11.0,5.0,1.0,6.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.000000,0.000000,0.000000,0.937031,0.832238,0.598121,0.638308,0.0,108,1
123159,2019-04-03,3,827f6afef3,1.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,3018.0,2958.0,3.0,2.0,2.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3018.0,2958.0,3.0,2.0,...,0.0,0.0,0.0,0.0,3.0,4451.0,2958.0,14.0,5.0,12.0,5.0,1.0,6.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,5.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.980116,0.980116,0.756326,0.771668,0.771668,0.678048,0.878677,0.0,110,0
155567,2019-04-04,2,827f6afef3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2958.0,2958.0,2.0,2.0,1.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3018.0,2958.0,3.0,2.0,...,0.0,0.0,0.0,0.0,3.0,4451.0,2958.0,14.0,5.0,12.0,5.0,1.0,6.0,

### Соберем строки для каждого email'а в одну

In [10]:
train_prepared[train_prepared['email'] == '30e12de274'].sort_values(by=['cut_date'])

,cut_date,days_to_end,email,num_country_max_1days,num_city_max_1days,android_max_1days,smarttv_max_1days,iphone_max_1days,ipad_max_1days,apple_max_1days,pc_max_1days,time_spent_sum_3days,time_spent_max_3days,num_title_ep_sum_3days,num_title_ep_max_3days,num_title_ru_sum_3days,num_title_ru_max_3days,num_country_max_3days,num_city_max_3days,other_content_sum_3days,top_1_sum_3days,top_2_sum_3days,top_3_sum_3days,top_4_sum_3days,top_5_sum_3days,top_6_sum_3days,top_7_sum_3days,top_8_sum_3days,top_9_sum_3days,top_10_sum_3days,android_max_3days,smarttv_max_3days,iphone_max_3days,ipad_max_3days,apple_max_3days,pc_max_3days,time_spent_sum_7days,time_spent_max_7days,num_title_ep_sum_7days,num_title_ep_max_7days,...,smarttv_max_21days,iphone_max_21days,ipad_max_21days,apple_max_21days,pc_max_21days,time_spent_sum_30days,time_spent_max_30days,num_title_ep_sum_30days,num_title_ep_max_30days,num_title_ru_sum_30days,num_title_ru_max_30days,num_country_max_30days,num_city_max_30days,other_content_sum_30days,top_1_sum_30days,top_2_sum_30days,top_3_sum_30days,top_4_sum_30days,top_5_sum_30days,top_6_sum_30days,top_7_sum_30days,top_8_sum_30days,top_9_sum_30days,top_10_sum_30days,android_max_30days,smarttv_max_30days,iphone_max_30days,ipad_max_30days,apple_max_30days,pc_max_30days,activity_1to3,activity_1to7,activity_1to14,activity_7to14,activity_7to21,activity_7to30,activity_14to30,label,days_use,days_no_use
4068,2019-03-29,2,30e12de274,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,9892.0,6425.0,3.0,1.0,3.0,1.0,1.0,7.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,15748.0,6425.0,6.0,1.0,...,0.0,0.0,0.0,0.0,7.0,37668.0,6425.0,37.0,8.0,19.0,1.0,1.0,13.0,0.0,35.0,0.0,18.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.086939,0.054610,0.027185,0.497803,0.487630,0.418074,0.839837,0.0,32,0
164160,2019-03-30,1,30e12de274,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,4980.0,2607.0,3.0,1.0,3.0,1.0,1.0,5.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,15280.0,6425.0,6.0,1.0,...,0.0,0.0,0.0,0.0,7.0,36969.0,6425.0,36.0,8.0,19.0,1.0,1.0,13.0,0.0,37.0,0.0,16.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.303815,0.099018,0.045644,0.460963,0.451964,0.413319,0.896643,0.0,33,0
113254,2019-03-31,0,30e12de274,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2433.0,1513.0,3.0,1.0,3.0,1.0,1.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,13872.0,6425.0,6.0,1.0,...,0.0,0.0,0.0,0.0,7.0,37029.0,6425.0,37.0,8.0,20.0,1.0,1.0,13.0,0.0,38.0,0.0,16.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.024661,0.004325,0.002115,0.489070,0.412513,0.374625,0.765994,0.0,34,0
77612,2019-04-17,14,30e12de274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,24.0,84044.0,29092.0,49.0,12.0,23.0,3.0,1.0,24.0,13.0,55.0,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,24.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.526450,0.0,42,9
138547,2019-04-18,13,30e12de274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,24.0,82210.0,29092.0,48.0,12.0,22.0,3.0,1.0,24.0,13.0,53.0,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,24.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.478117,0.0,42,10


In [0]:
def combine(df, features):

    dict_df = {'email': [df['email'].values[0]], 'num_dots': [df.shape[0]], 'label': [df['label'].values[-1]]}

    day_of_observ = (df['cut_date'] - df['cut_date'].iloc[0]).dt.days.values

    # days from last prolongation:
    days_to_end = df['days_to_end'].values
    days_current = days_to_end[0]
    days_from_prolong = day_of_observ[-1] - day_of_observ[0]
    for i in range(1, len(days_to_end)):
        if days_to_end[i] > days_current:
            days_from_prolong = day_of_observ[-1] - day_of_observ[i]
        days_current = days_to_end[i]
    dict_df['days_from_prolong'] = [days_from_prolong]

    for feature in features:

        y = df[feature].values.reshape([-1, 1])
        dict_df[feature + '_mean'] = [y.mean()]

        if df.shape[0] > 1:
            reg = LinearRegression().fit(day_of_observ.reshape([-1, 1]), y)
            R2 = reg.score(day_of_observ.reshape([-1, 1]), y)
            dict_df[feature + '_trend'] = [reg.coef_[0][0]]
            dict_df[feature + '_R2'] = [R2]
        else:
            dict_df[feature + '_trend'] = [0]
            dict_df[feature + '_R2'] = [-1]
    
    df_email = pd.DataFrame(data=dict_df)
    return df_email

In [0]:
def create_df(df, features):
    """
    Создадим датафрейм, в котором каждому email'у будет соответствовать 1 строка
    """
    emails = df['email'].unique()
    new_dfs = []
    for i, email in enumerate(emails):
        if i % 1000 == 0:
            print(i)
        df_email = combine(df[df['email'] == email].sort_values(by=['cut_date']), features)
        new_dfs.append(df_email)
    
    new_df = pd.concat(new_dfs)
    return new_df

In [0]:
features = [col for col in train_prepared.columns if col not in ['cut_date', 'email', 'label']]

# Train dataset
new_df = create_df(train_prepared, features)

In [0]:
# Test dataset
new_df_test = create_df(test_prepared, features)

In [18]:
# Save new train dataset

path_df = 'data/train_prepared_v1.csv'
new_df.to_csv(path_df, index=False)

new_df.head()

,email,num_dots,label,days_from_prolong,days_to_end_mean,days_to_end_trend,days_to_end_R2,num_country_max_1days_mean,num_country_max_1days_trend,num_country_max_1days_R2,num_city_max_1days_mean,num_city_max_1days_trend,num_city_max_1days_R2,android_max_1days_mean,android_max_1days_trend,android_max_1days_R2,smarttv_max_1days_mean,smarttv_max_1days_trend,smarttv_max_1days_R2,iphone_max_1days_mean,iphone_max_1days_trend,iphone_max_1days_R2,ipad_max_1days_mean,ipad_max_1days_trend,ipad_max_1days_R2,apple_max_1days_mean,apple_max_1days_trend,apple_max_1days_R2,pc_max_1days_mean,pc_max_1days_trend,pc_max_1days_R2,time_spent_sum_3days_mean,time_spent_sum_3days_trend,time_spent_sum_3days_R2,time_spent_max_3days_mean,time_spent_max_3days_trend,time_spent_max_3days_R2,num_title_ep_sum_3days_mean,num_title_ep_sum_3days_trend,num_title_ep_sum_3days_R2,...,smarttv_max_30days_R2,iphone_max_30days_mean,iphone_max_30days_trend,iphone_max_30days_R2,ipad_max_30days_mean,ipad_max_30days_trend,ipad_max_30days_R2,apple_max_30days_mean,apple_max_30days_trend,apple_max_30days_R2,pc_max_30days_mean,pc_max_30days_trend,pc_max_30days_R2,activity_1to3_mean,activity_1to3_trend,activity_1to3_R2,activity_1to7_mean,activity_1to7_trend,activity_1to7_R2,activity_1to14_mean,activity_1to14_trend,activity_1to14_R2,activity_7to14_mean,activity_7to14_trend,activity_7to14_R2,activity_7to21_mean,activity_7to21_trend,activity_7to21_R2,activity_7to30_mean,activity_7to30_trend,activity_7to30_R2,activity_14to30_mean,activity_14to30_trend,activity_14to30_R2,days_use_mean,days_use_trend,days_use_R2,days_no_use_mean,days_no_use_trend,days_no_use_R2
0,8ba752f2c5,13,0.0,12,8.0,-1.0,1.0,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,1.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,1.0,0.000000,0.000000,1.000000,14.615385,-1.747253,0.652851,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,31.000000,0.000000,1.000000,23.000000,1.000000,1.000000
0,752a6d96f7,13,1.0,12,8.0,-1.0,1.0,0.076923,-0.016484,0.053571,0.076923,-0.016484,0.053571,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,1.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.076923,-0.016484,0.053571,13.846154,-1.978022,0.085714,13.846154,-1.978022,0.085714,0.230769,-0.032967,0.085714,...,1.0,11.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.0,0.0,1.0,2.153846,-0.329670,0.714286,0.076910,-0.016481,0.053571,0.025640,-0.005494,0.053571,0.000447,-0.000096,0.053571,0.133126,-0.024739,0.112225,0.044818,-0.017711,0.227203,0.030419,-0.012141,0.230987,0.299731,-0.046753,0.538105,153.615385,0.494505,0.535714,4.384615,0.505495,0.546629
0,827f6afef3,9,0.0,8,4.0,-1.0,1.0,0.222222,0.000000,0.000000,0.444444,0.033333,0.008108,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,1.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.444444,0.033333,0.008108,1098.555556,240.266667,0.210590,1091.888889,239.266667,0.211765,1.222222,0.066667,0.027907,...,1.0,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.0,0.0,1.0,5.333333,-0.350000,0.525000,0.219994,-0.000329,0.000004,0.115897,0.015286,0.016617,0.091032,0.011556,0.015980,0.891518,-0.038054,0.815113,0.822485,-0.015135,0.630290,0.656934,0.017526,0.784592,0.754491,0.053024,0.851980,107.222222,1.083333,0.823052,1.777778,-0.083333,0.026786
0,346e0f766c,14,1.0,13,6.5,-1.0,1.0,0.214286,-0.072527,0.507692,0.500000,-0.173626,0.508018,0.0,0.0,1.0,0.0,0.0,1.0,0.5,-0.173626,0.508018,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,647.642857,-210.217582,0.637936,453.214286,-142.279121,0.646535,1.571429,-0.501099,0.668691,...,1.0,11.571429,-0.145055,0.507692,0.000000,0.000000,1.000000,0.0,0.0,1.0,4.214286,-0.494505,0.692308,0.050662,-0.017221,0.235225,0.027776,-0.009549,0.241231,0.015416,-0.005320,0.253762,0.256476,-0.057913,0.8936

In [0]:
all_emails = train_prepared['email'].unique()

emails_train = all_emails[:14000]
emails_test = all_emails[14000:]

X_train = new_df[new_df['email'].isin(emails_train)].drop(['label', 'email'], axis=1)
y_train = new_df[new_df['email'].isin(emails_train)]['label']

X_test = new_df[new_df['email'].isin(emails_test)].drop(['label', 'email'], axis=1)
y_test = new_df[new_df['email'].isin(emails_test)]['label']

In [27]:
clf = LGBMClassifier(n_estimators=300, learning_rate=0.1)
clf.fit(X_train, y_train)
    
predictions = clf.predict_proba(X_test)
auc_score = roc_auc_score(y_test, predictions[:, 1])
print(auc_score)

0.7181675664338641


### Подберем гиперпараметры классификатора

In [0]:
params = {
    'n_estimators': list(range(50, 300, 50)),
    'num_leaves': [7, 15, 23],
    'max_depth': [5, 7, 9]
}

all_emails = train_prepared['email'].unique()

emails_train = all_emails[:14000]
emails_test = all_emails[14000:]

X_train = train_prepared[train_prepared['email'].isin(emails_train)].drop(['label', 'email', 'cut_date'], axis=1)
y_train = train_prepared[train_prepared['email'].isin(emails_train)]['label']

X_test = train_prepared[train_prepared['email'].isin(emails_test)].drop(['label', 'email', 'cut_date'], axis=1)
y_test = train_prepared[train_prepared['email'].isin(emails_test)]['label']

In [52]:
all_results = {}

for n_estimators in params['n_estimators']:
    for num_leaves in params['num_leaves']:
        for max_depth in params['max_depth']:
            
            clf = LGBMClassifier(n_estimators=n_estimators, num_leaves=num_leaves, max_depth=max_depth, learning_rate=0.1)
            clf.fit(X_train, y_train)
    
            predictions = clf.predict_proba(X_test)
            auc_score = roc_auc_score(y_test, predictions[:, 1])
            
            print(f'({n_estimators}, {num_leaves}, {max_depth}): {auc_score}')
            all_results[(n_estimators, num_leaves, max_depth)] = auc_score

(50, 7, 5): 0.6535429657888215
(50, 7, 7): 0.6544731106770074
(50, 7, 9): 0.6544731106770074
(50, 15, 5): 0.654188630259053
(50, 15, 7): 0.6548777784965596
(50, 15, 9): 0.6538956808706047
(50, 23, 5): 0.6568344318546271
(50, 23, 7): 0.6548081885241122
(50, 23, 9): 0.6563769662225203
(100, 7, 5): 0.6542710717461659
(100, 7, 7): 0.6561628775730364
(100, 7, 9): 0.6561628775730364
(100, 15, 5): 0.6535099008141874
(100, 15, 7): 0.6538236513426439
(100, 15, 9): 0.6528532890948452
(100, 23, 5): 0.6566972677889381
(100, 23, 7): 0.6540161142778005
(100, 23, 9): 0.6568370285803837
(150, 7, 5): 0.6537486583583592
(150, 7, 7): 0.6563906514228235
(150, 7, 9): 0.6563906514228235
(150, 15, 5): 0.6517137000517522
(150, 15, 7): 0.6538837404877835
(150, 15, 9): 0.6542309363884191
(150, 23, 5): 0.656420463656773
(150, 23, 7): 0.6527602989788035
(150, 23, 9): 0.6555719926271212
(200, 7, 5): 0.654075975639979
(200, 7, 7): 0.6552652441468889
(200, 7, 9): 0.6552652441468889
(200, 15, 5): 0.6519137709378098
(

In [53]:
sorted(all_results.items(), key=lambda item: item[1], reverse=True)

[((100, 23, 9), 0.6568370285803837),
 ((50, 23, 5), 0.6568344318546271),
 ((100, 23, 5), 0.6566972677889381),
 ((150, 23, 5), 0.656420463656773),
 ((150, 7, 7), 0.6563906514228235),
 ((150, 7, 9), 0.6563906514228235),
 ((50, 23, 9), 0.6563769662225203),
 ((100, 7, 7), 0.6561628775730364),
 ((100, 7, 9), 0.6561628775730364),
 ((200, 23, 9), 0.655681467396058),
 ((150, 23, 9), 0.6555719926271212),
 ((200, 7, 7), 0.6552652441468889),
 ((200, 7, 9), 0.6552652441468889),
 ((200, 23, 5), 0.655005762908916),
 ((250, 7, 7), 0.6549558829960931),
 ((250, 7, 9), 0.6549558829960931),
 ((50, 15, 7), 0.6548777784965596),
 ((50, 23, 7), 0.6548081885241122),
 ((200, 15, 9), 0.6545655791700682),
 ((250, 23, 9), 0.6545648662093999),
 ((50, 7, 7), 0.6544731106770074),
 ((50, 7, 9), 0.6544731106770074),
 ((100, 7, 5), 0.6542710717461659),
 ((150, 15, 9), 0.6542309363884191),
 ((250, 7, 5), 0.6541992403893813),
 ((50, 15, 5), 0.654188630259053),
 ((200, 7, 5), 0.654075975639979),
 ((100, 23, 7), 0.65401611

In [54]:
clf = LGBMClassifier(n_estimators=600, learning_rate=0.1, num_leaves=23, max_depth=5)

clf.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='auc',
    verbose=True,
    early_stopping_rounds=200,
)

[1]	valid_0's binary_logloss: 0.567982	valid_0's auc: 0.640495
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.563415	valid_0's auc: 0.64214
[3]	valid_0's binary_logloss: 0.559884	valid_0's auc: 0.642667
[4]	valid_0's binary_logloss: 0.557054	valid_0's auc: 0.642827
[5]	valid_0's binary_logloss: 0.554779	valid_0's auc: 0.642598
[6]	valid_0's binary_logloss: 0.552789	valid_0's auc: 0.645265
[7]	valid_0's binary_logloss: 0.551236	valid_0's auc: 0.64548
[8]	valid_0's binary_logloss: 0.550056	valid_0's auc: 0.646674
[9]	valid_0's binary_logloss: 0.549136	valid_0's auc: 0.647097
[10]	valid_0's binary_logloss: 0.548247	valid_0's auc: 0.647667
[11]	valid_0's binary_logloss: 0.54765	valid_0's auc: 0.647932
[12]	valid_0's binary_logloss: 0.547132	valid_0's auc: 0.648463
[13]	valid_0's binary_logloss: 0.546813	valid_0's auc: 0.648885
[14]	valid_0's binary_logloss: 0.546456	valid_0's auc: 0.649173
[15]	valid_0's binary_logloss: 0.546356	valid_0's auc

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=600, n_jobs=-1, num_leaves=23, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

### Получим результат для тестового датасета и сохраним результат

In [0]:
test_data = test_prepared.drop(['email', 'cut_date'], axis=1)

best_clf = LGBMClassifier(n_estimators=50, learning_rate=0.1, num_leaves=23, max_depth=5)
best_clf.fit(train_prepared.drop(['label', 'email', 'cut_date'], axis=1), train_prepared['label'])
test_predictions = best_clf.predict_proba(test_data)

In [0]:
df_predictions = pd.DataFrame({
        'email': test_prepared.email,
        'prediction': test_predictions[:, 1]
    })

In [0]:
# Save predictions

path_submission = 'data/test_submit.csv'
df_predictions.to_csv(path_submission, index=False)